In [4]:
import cv2
import mediapipe as mp
import time
import numpy as np
import pyautogui

mp_hands = mp.solutions.hands
hands = mp_hands.Hands()

I0000 00:00:1742328309.252133 3350287 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M1 Pro


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1742328309.263178 3351471 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1742328309.268637 3351471 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [5]:
   
def findHands(hands, image):
    return hands.process(image)
        
def drawHands(image, my_hands):
    global mp_hands
    if my_hands.multi_hand_landmarks:
        for hand_landmarks in my_hands.multi_hand_landmarks:
            mp.solutions.drawing_utils.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)            
    return image

def getPointer(image, my_hands):  # image is only used for its shape
    if my_hands.multi_hand_landmarks:
        for hand_landmarks in my_hands.multi_hand_landmarks:
            # check if landmark is tip of index
            if hand_landmarks.landmark[8].y < hand_landmarks.landmark[5].y:
                pos_idx = (int(hand_landmarks.landmark[8].x*image.shape[1]), int(hand_landmarks.landmark[8].y*image.shape[0]))
            else:
                pos_idx = (int(hand_landmarks.landmark[8].x*image.shape[1]), int(hand_landmarks.landmark[8].y*image.shape[0]))
            # check if landmark is tip of thumb
            if hand_landmarks.landmark[4].x < hand_landmarks.landmark[3].x:
                pos_thumb = (int(hand_landmarks.landmark[4].x*image.shape[1]), int(hand_landmarks.landmark[4].y*image.shape[0]))
            else:
                pos_thumb = (int(hand_landmarks.landmark[4].x*image.shape[1]), int(hand_landmarks.landmark[4].y*image.shape[0]))
        return pos_idx, pos_thumb
    return None, None

def getKeypoints(myhands):
    data = []
    if myhands.multi_hand_landmarks:
        for hand_landmarks in myhands.multi_hand_landmarks:
            for i, landmark in enumerate(hand_landmarks.landmark):
                data.append([landmark.x, landmark.y, landmark.z])
    data = np.array(data)
    
    hands = []
    centers = []
    for i in range(0, data.shape[0], 21):
        hands.append(data[i:i+21])
        centers.append(np.mean(hands[-1], axis = 0).reshape(3))
    hands = np.array(hands).reshape(-1, 21, 3)
    centers = np.array(centers).reshape(-1, 3)
        
    if len(centers.shape) == 2 and centers.shape[0] >= 2:
        dist = (np.sum((centers[0, :2] - centers[1, :2])**2))**0.5
    else:
        dist = None
    
    dataset = {
        "hands": hands,
        "centers": centers,
        "distance": dist
    }
    return dataset


cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1920//4)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 1080//4)
dataset = []

prev = time.time()
history = []


OpenCV: not authorized to capture video (status 0), requesting...
OpenCV: camera failed to properly initialize!


In [6]:
import multiprocessing 

def asyncload(result_queue):
    ret, frame = cap.read()
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image = cv2.flip(image, 1)


    # Find keypoints, draw them and get back useful data
    myhands = findHands(hands, image)
    drawHands(image, myhands)
    data = getKeypoints(myhands)
    pol, ind = getPointer(image, myhands)

    result_queue.put([image, pol, ind, data])


In [7]:
import threading
import queue

In [8]:
result_queue = queue.Queue()
thread1 = threading.Thread(target=asyncload, args=(result_queue,))
thread1.start()

while True:
    thread1.join()
    image, pol, ind, data = result_queue.get()
    
    thread1 = threading.Thread(target=asyncload, args=(result_queue,))
    thread1.start()


    if image.mean() < 90:
        # throw an exception
        print("No light")
        break

    
    if pol is not None and ind is not None:
        mean = (pol[0]+ind[0])/2, (pol[1]+ind[1])/2 
        dist = ((pol[0]-ind[0])**2 + (pol[1]-ind[1])**2)**0.5

        if dist < 50:
            cv2.circle(image, (int(mean[0]), int(mean[1])), 10, (255, 255, 255), -1)
            pyautogui.moveTo(mean[0]*4, mean[1]*4)
            pyautogui.click()

        elif dist > 50 and dist < 250:
            cv2.circle(image, (int(mean[0]), int(mean[1])), 10, (0, 0, 0), -1)
            pyautogui.moveTo(mean[0]*4, mean[1]*4)

    
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    image = cv2.resize(image, (1280, 720))
    
    # find centers
    if len(data["centers"]) > 0:
        for x, y, z in data["centers"]:
            image = cv2.circle(image, (int(x*1280), int(y*720)), 10, (255, 255, 255), -1)

    n = 0
    
    cv2.putText(image, str(round(1/(time.time()-prev), 1)), (20, 50+50*(n+2)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    cv2.imshow('Hand Tracking', cv2.resize(image, (1600//2, 900//2)))

    # Step 10: Break the loop if the user presses the 'q' key
    if cv2.waitKey(10) & (0xFF == ord('q') or 0xFF == 27):
        break    

    prev = time.time()
    
    


Exception in thread Thread-9:
Traceback (most recent call last):
  File "/Applications/Xcode.app/Contents/Developer/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/threading.py", line 973, in _bootstrap_inner
    self.run()
  File "/Users/lucam/Library/Python/3.9/lib/python/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/Applications/Xcode.app/Contents/Developer/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/threading.py", line 910, in run
    self._target(*self._args, **self._kwargs)
  File "/var/folders/8f/s2460vq56slb0jp9q_sc8ksw0000gn/T/ipykernel_55271/3356976369.py", line 5, in asyncload
cv2.error: OpenCV(4.11.0) /Users/xperience/GHA-Actions-OpenCV/_work/opencv-python/opencv-python/opencv/modules/imgproc/src/color.cpp:199: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'



KeyboardInterrupt: 

In [ ]:



# Step 9: Release the webcam and close the window
cap.release()
cv2.destroyAllWindows()

